In [ ]:
import ee
#import ipyleaflet
import geemap
ee.Initialize(project='ee-experiments')

In [ ]:
CDL = ee.ImageCollection('USDA/NASS/CDL').select('cropland').filterDate('2022-01-01', '2022-12-30').first()
USA_COUNTIES = ee.FeatureCollection("TIGER/2018/Counties")
AOI = USA_COUNTIES.filter(ee.Filter.eq('GEOID', '19073'));

In [ ]:
def sld_cdl_ramp():
    cdl_info = CDL.getInfo()
    sld_ramp ='<RasterSymbolizer><ColorMap type="values" extended="true" >'
    for color, crop_id, crop_name in zip(
        cdl_info['properties']['cropland_class_palette'],
        cdl_info['properties']['cropland_class_values'],
        cdl_info['properties']['cropland_class_names']
    ):
        sld_ramp += f'<ColorMapEntry color="#{color}" quantity="{crop_id}" label="{crop_name}" />'
    sld_ramp += '</ColorMap></RasterSymbolizer>'
    return sld_ramp

def sld_three_class_cdl_ramp():
    cdl_info = CDL.getInfo()
    sld_ramp ='<RasterSymbolizer><ColorMap type="values" extended="true" >'
    for color, crop_id, crop_name in zip(
        cdl_info['properties']['cropland_class_palette'],
        cdl_info['properties']['cropland_class_values'],
        cdl_info['properties']['cropland_class_names']
    ):
        if crop_name.lower() in ('corn', 'soybeans'):
            sld_ramp += f'<ColorMapEntry color="#{color}" quantity="{crop_id}" label="{crop_name}" />'

    # put everything other than Corn and Soy in the "other" class, and color blue
    sld_ramp += '<ColorMapEntry color="#00a8e2" quantity="3" label="other" />'
    sld_ramp += '</ColorMap></RasterSymbolizer>'
    return sld_ramp

In [ ]:
Map = geemap.Map()
Map.center_object(AOI, 10)
Map.addLayer(AOI, {}, 'Greene County IA');
Map.addLayer(ee.Image(CDL).sldStyle(sld_cdl_ramp()).clip(AOI), {}, 'CDL orig');

# remap cdl into corn 1, soy 2, and everything else to 3
CDL_REMAP = CDL.remap([1, 5], [1, 2], 3).rename('crop_type');
# print(cdl_remap);
Map.addLayer(ee.Image(CDL_REMAP).sldStyle(sld_three_class_cdl_ramp()).clip(AOI), {}, 'CDL corn/soy/other');

Map